In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [3]:
def CreateEncoder(Shape):
    Inputs = Input(shape=Shape)

    #PTEncoder = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', pooling='avg') #pre_trained_encoder.trainable = False
    PTEncoder = tf.keras.applications.VGG16(include_top=False, weights='imagenet', pooling='avg')

    # Connect the encoder with the inputs
    x = PTEncoder(Inputs)

    # Define additional layers
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)

    # Define model
    encoder = Model(inputs=Inputs, outputs=x)

    return encoder

In [4]:
def CreateDecoder(Shape):
    Inputs = Input(Shape)

    # Define dense layers
    x = Dense(512, activation='relu')(Inputs)
    x = Dense(7 * 7 * 128, activation='relu')(x)

    # Reshape the dense output to match the shape of the last convolutional layer in the encoder
    x = tf.reshape(x, (-1, 7, 7, 128))

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    x = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Define model
    decoder = Model(inputs=Inputs, outputs=x)

    return decoder

In [5]:
def CreateAutoencoder(encoder, decoder, Shape):
    Inputs = Input(shape=Shape)

    # Encode the input image
    encoded = encoder(Inputs)

    # Decode the encoded representation
    decoded = decoder(encoded)

    # Define autoencoder model
    Autoencoder = Model(inputs=Inputs, outputs=decoded)

    return Autoencoder

In [6]:
def CreatePath(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [7]:
def SaveImage(ImagePath, ImageName, NImagesPath):
    InputImage = Image.open(ImagePath)
    InputImagePath = os.path.join(NImagesPath, ImageName + '.jpg')
    InputImage.save(InputImagePath)

In [8]:
def CreateFigure(ImagePath, SimilarImages, SimilarityPercentages, TotalPercentageSimilarity, distances):
    fig, ax = plt.subplots(nrows=1, ncols=len(SimilarImages) + 1, figsize=(15, 5))

    InputImage = Image.open(ImagePath)
    ax[0].imshow(InputImage)
    #ax[0].set_title(f'Input Image\n{TotalPercentageSimilarity:.2f}%')
    ax[0].axis('off')

    for i, img in enumerate(SimilarImages):
        img = Image.open(img)
        img = img.resize(InputImage.size)
        ax[i+1].set_title(f'{SimilarityPercentages[i]:.2f}%')
        #ax[i+1].set_title(f'{distances[i]:.2f}')
        ax[i+1].imshow(img)
        ax[i+1].axis('off')

    return fig

In [9]:
def ShowSimilarImages(ImagePath, SimilarImagesPath, SimilarImages, SimilarityPercentages, TotalPercentageSimilarity, distances):
  
    if not SimilarImages:
        print('No similar images found.')
    else:
        CreatePath(SimilarImagesPath)
        SaveImage(ImagePath, 'InputImage', SimilarImagesPath)

        for i, p in enumerate(SimilarImages):
            #SaveImage(p, f'SimilarImage{i+1}_{SimilarityPercentages[i]:.2f}', SimilarImagesPath)
            SaveImage(p, f'SimilarImage{i+1}_{distances[i]:.2f}', SimilarImagesPath)

        fig = CreateFigure(ImagePath, SimilarImages, SimilarityPercentages, TotalPercentageSimilarity, distances)
        plt.show()

In [10]:
def Preprocess(img, Shape):
    img = img.resize(Shape)
    img = img.convert('RGB')
    img = np.array(img) / 255.0
    
    img = tf.expand_dims(img, axis=0)
   
    return img

In [11]:
def LoadDataset(path, Shape):
    Images = []

    for root, dirs, files in os.walk(path):
        for dir in dirs:
            for file in os.listdir(os.path.join(root, dir)):

                if file.endswith('.jpg') or file.endswith('.png'):

                    img = Image.open(os.path.join(root, dir, file))

                    if img is not None:
                        img = tf.keras.preprocessing.image.array_to_img(img)
                        img = Preprocess(img, Shape)
                        Images.append(img)
    
    if Images:
        Images = np.concatenate(Images)

    return Images

In [12]:
def TrainAutoencoder(Encoder, Autoencoder, Images, batch_size, epochs):
    # Compile the autoencoder
    Autoencoder.compile(optimizer='adam', loss='mean_squared_error')

    # Train the autoencoder
    Autoencoder.fit(Images, Images, batch_size=batch_size, epochs=epochs)

    return Autoencoder

In [13]:
def LoadModel(DataPath, ModelPath, Shape, Encoder, Autoencoder, batch_size, epochs):
    if os.path.exists(ModelPath):
        Autoencoder = tf.keras.models.load_model(ModelPath)
            
    else:
        Images = LoadDataset(DataPath, Shape)
        Autoencoder = TrainAutoencoder(Encoder, Autoencoder, Images, batch_size, epochs)
        
        Autoencoder.save(ModelPath)
        
    return Autoencoder

In [14]:
def EuclideanDistance(img1, img2):
  SquaredDifference = np.square(img1 - img2)
  SummedSquaredDifference = np.sum(SquaredDifference)
  Distance = np.sqrt(SummedSquaredDifference)

  return Distance

In [15]:
def Search(path, ImagePath, Shape, Encoder, n_neighbors):
    img = Image.open(ImagePath)
    img = Preprocess(img, Shape)
    
    img_embedding = Encoder.predict(img)

    SimilarImages = []
    SimilarityPercentages = []
    distances = []

    for root, dirs, files in os.walk(path):
      for dir in dirs:
        FilePath = os.path.join(root, dir)
        FileList = os.listdir(FilePath)

        if not FileList:
          print('Error: No files found in directory.', FilePath)
          continue

        for File in FileList:
            SimilarImages.append(os.path.join(FilePath, File))

            img2 = Image.open(os.path.join(FilePath, File))
            img2 = Preprocess(img2, Shape)
            img2_embedding = Encoder.predict(img2)

            distance = EuclideanDistance(img_embedding, img2_embedding)
            distances.append(distance)

            #SimilarityPercentage = (1 - distance) * 100
            #SimilarityPercentages.append(SimilarityPercentage)

    MDistance = np.max(distances)

    SortedSimilarity = sorted(zip(distances, SimilarImages), reverse=False) #SortedSimilarity = sorted(zip(SimilarityPercentages, SimilarImages), reverse=True)
    distances, SimilarImages = zip(*SortedSimilarity) #SimilarityPercentages, SimilarImages = zip(*SortedSimilarity)

    SimilarImages = list(SimilarImages)[:n_neighbors]
    distances = list(distances)[:n_neighbors] #SimilarityPercentages = list(SimilarityPercentages)[:n_neighbors]

    SimilarityPercentages = [(1 - distance / MDistance) * 100 for distance in distances]
    #SimilarityPercentages = [round((1 - distance) * 100, 2) for distance in distances[0]]
    
    TotalPercentageSimilarity = np.round(np.sum(SimilarityPercentages) / len(SimilarityPercentages), 2)
    #TotalPercentageSimilarity = round(sum([(1 - distance) for distance in distances[0]]) * 100 / len(distances[0]), 2)
    #TotalPercentageSimilarity = sum(SimilarityPercentages) / len(SimilarityPercentages)
    
    return SimilarImages, SimilarityPercentages, TotalPercentageSimilarity, distances

In [16]:
cwd = os.getcwd()
DataPath = os.path.join(cwd, 'drive', 'MyDrive', 'Colab Notebooks', 'archive', 'Images')
ModelPath = os.path.join(cwd, 'drive', 'MyDrive', 'Colab Notebooks', 'archive', 'SavedModel', 'Model.h5')
SimilarImagesPath = os.path.join(cwd, 'drive', 'MyDrive', 'Colab Notebooks', 'archive', 'SavedModel', 'SimilarImages')

In [17]:
image_width = 56
image_height = 56

Shape = (image_width, image_height, 3)
SPreprocess = (image_width, image_height)

batch_size = 32
epochs = 10

n_neighbors = 5

In [18]:
Encoder = CreateEncoder(Shape)
Decoder = CreateDecoder(Encoder.layers[-1].output_shape[1:])
Autoencoder = CreateAutoencoder(Encoder, Decoder, Shape)

In [ ]:
Autoencoder = LoadModel(DataPath, ModelPath, SPreprocess, Encoder, Autoencoder, batch_size, epochs)

In [ ]:
ImagePath = os.path.join(cwd, 'drive', 'MyDrive', 'Colab Notebooks', 'Dog.jpg')

SimilarImages, SimilarityPercentages, TotalPercentageSimilarity, distances = Search(DataPath, ImagePath, SPreprocess, Encoder, n_neighbors)

ShowSimilarImages(ImagePath, SimilarImagesPath, SimilarImages, SimilarityPercentages, TotalPercentageSimilarity, distances)